In [13]:
import sys
import json
import re

In [62]:
lib_hashtags = set([ht.lower() for ht in [
    'resist',
    'impeachtrump',
    'p2',
    'bluewave2018'
    'fucktrump',
    'resistance',
    'bluewave',
    'fbr', 
    'flipitblue',
    'votebluetosaveamerica',
    'votewithbeto', 
    'Votebeto',
    'betofortexas',
    'gopvotingblue',
    'resisttrump',
    'iamawakenow', 
    'votethemout2018',
    'votebluetoendthisnightmare',
    'trumplies',
    'trumpliesmatter', 
    'goplies',
    'factsmatter',
    'flipthe49th',
    'thisisnotnormal',
    'takeitback',
    'blacklivesmatter',
    'dumptrump',
    'antitrump',
    'protectourcare',
    'votedem',
    'FlipTheHouse',
    'FlipTheSenate',
    'RejectTheNRA',
    'VoteBlue',
    'BlueWave2018',
    'GunSense',
    'flipthe45th',
    'VoteBlueToSaveHealthcare',
    'OpenBorders',
    'VoteBlue2018',
    'CABlueWAVE',
    'VoteBlueThisTuesday',
    'VoteBLUENoMatterWho',
    'Beto4Texas',
    'VoteBlueToSaveAmerica',
    'VoterSuppressionTactics',
    'AbolishIce',
]])
cons_hashtags = set([ht.lower() for ht in [
    'trumptrain',
    'maga',
    'walkaway',
    'qanon',
    'greatawakening',
    'americafirst',
    'wwg1wga',
    'tcot',
    'kag',
    'patriotsunited',
    'qarmy',
    'ccot',
    'thegreatawakening',
    'q',
    'trump2020',
    'makeamericagreatagain',
    'liberalismisamentaldisorder',
    'trumpsarmy',
    'voteredtosaveamerica',
    'votered',
    'unitedvotered',
    'supportice',
    'keepredstatesred',
    'turnbluestatesred',
    'voteredtosavecalifornia',
    'MadMaxine',
    'VoteRedToSaveAmerica2018',
    'ResistTheResistance',
    'VoteAntonioToSaveCalifornia',
    'VoteRed2018',
    'GetMaxineOut',
    'RedWave2018',
    'blexit',
    'PledgeToVoteRed',
    'MAGA2020',
    'VoteRedSaveAmerica2018',
    'RedWave2018andBeyond',
    'redcalifornia',
    'VoteDemsOut',
    'MakeAmericsGreatAgain',
    'REDWAVECOMING',
    'RedTsunami',
    'VoteAntonioSabato',
    'VoteRedOrAmericaIsDead',
    'RedWave',
    'VoteRedSaveAmerica',
    'VoteRedMidterms2018',
    'CaravanInvasion',
]])


In [3]:
infile = 'candidate_handles1A2018-11-02.json'

In [22]:
with open('candidate_handles.txt') as f:
    partisan_tweets_per_candidate = {}
    handles = []
    for line in f:
        partisan_tweets_per_candidate[line[:-1].lower()] = {'liberal': 0, 'conservative': 0, 'neutral': 0}
        handles.append(line[:-1].lower())

In [17]:
handles

['@kennedy4alabama',
 '@repbyrne',
 '@tabithak',
 '@repmartharoby',
 '@haganforhouse',
 '@repmikerogersal',
 '@leeauman2018',
 '@robert_aderholt',
 '@peter_joffrion',
 '@repmobrooks',
 '@votedannerkline',
 '@usrepgarypalmer',
 '@repterrisewell',
 '@repdonyoung',
 '@alysegalvin',
 '@gregoryfitch',
 '@jameshillsid',
 '@repohalleran',
 '@wendyrogersaz',
 '@dbrauer',
 '@ann_kirkpatrick',
 '@leapeterson',
 '@melissagrableaf',
 '@flayerjordan',
 '@repraulgrijalva',
 '@nicolasbpierson',
 '@hannahhdunawayy',
 '@drpaulgosar',
 '@brill4congress',
 '@andybiggs4az',
 '@joangreeneaz',
 '@davidschweikert',
 '@anitamalik',
 '@rubengallego',
 '@mrdanhadfield',
 '@debbielesko',
 '@hiral4congress',
 '@whoisaugie',
 '@stehum',
 '@gregstantonaz',
 '@steveferraramd',
 '@iurekiko777rex',
 '@reprickcrawford',
 '@decideondesai',
 '@elvispresley',
 '@repfrenchhill',
 '@clarketucker',
 '@joeswaff4cong',
 '@womack4congress',
 '@joshuamahony',
 '@drjoshmoody',
 '@jason_tate',
 '@westermanar',
 '@haydenshamel',
 '

In [18]:
partisan_tweets_per_candidate

{'@kennedy4alabama': {'conservative': 0, 'liberal': 0},
 '@repbyrne': {'conservative': 0, 'liberal': 0},
 '@tabithak': {'conservative': 0, 'liberal': 0},
 '@repmartharoby': {'conservative': 0, 'liberal': 0},
 '@haganforhouse': {'conservative': 0, 'liberal': 0},
 '@repmikerogersal': {'conservative': 0, 'liberal': 0},
 '@leeauman2018': {'conservative': 0, 'liberal': 0},
 '@robert_aderholt': {'conservative': 0, 'liberal': 0},
 '@peter_joffrion': {'conservative': 0, 'liberal': 0},
 '@repmobrooks': {'conservative': 0, 'liberal': 0},
 '@votedannerkline': {'conservative': 0, 'liberal': 0},
 '@usrepgarypalmer': {'conservative': 0, 'liberal': 0},
 '@repterrisewell': {'conservative': 0, 'liberal': 0},
 '@repdonyoung': {'conservative': 0, 'liberal': 0},
 '@alysegalvin': {'conservative': 0, 'liberal': 0},
 '@gregoryfitch': {'conservative': 0, 'liberal': 0},
 '@jameshillsid': {'conservative': 0, 'liberal': 0},
 '@repohalleran': {'conservative': 0, 'liberal': 0},
 '@wendyrogersaz': {'conservative': 

In [63]:
sys.stdout.write("parsing " + infile + "...")
sys.stdout.flush()

with open(infile) as f:
    for line in f:
        if line != None:
            tweet = json.loads(line.rstrip('\n'))
            try:
                # Classify tweet as liberal or conservative
                lib_score = 0
                cons_score = 0
                tweet_sentiment = 'neutral'
                for hashtag in tweet['entities']['hashtags']:
                    ht = hashtag['text'].lower()
                    if ht in lib_hashtags:
                        lib_score +=1
                    elif ht in cons_hashtags:
                        cons_score += 1
                if lib_score > cons_score:
                    tweet_sentiment = 'liberal'
#                         print(tweet['text'].lower())
                elif lib_score < cons_score:
                    tweet_sentiment = 'conservative'
                else:
                    if tweet['entities']['hashtags']:
                        print(tweet['entities']['hashtags'])

                # Get the candidates mentioned in tweets
                mentions = re.findall('@[a-zA-Z0-9_]{1,15}', tweet['text'].lower())
                # Update dictionary
                for person in mentions:
                    try:
                        partisan_tweets_per_candidate[person][tweet_sentiment] += 1
                    except:
                        pass;
#                     print(tweet_sentiment)
            except:
                print('FAILED')


parsing candidate_handles1A2018-11-02.json...[{'text': 'PittsburghShooting', 'indices': [85, 104]}]
[{'text': 'racism', 'indices': [83, 90]}]
[{'text': 'MarkLevin', 'indices': [41, 51]}]
[{'text': 'Hannity', 'indices': [90, 98]}]
[{'text': 'CA50', 'indices': [78, 83]}]
[{'text': 'AZ', 'indices': [115, 118]}]
[{'text': 'LSSC', 'indices': [78, 83]}]
[{'text': 'abortion', 'indices': [103, 112]}]
[{'text': 'gaslight', 'indices': [39, 48]}, {'text': 'cult45', 'indices': [54, 61]}]
[{'text': 'TakeonAmerica', 'indices': [45, 59]}]
[{'text': 'Vote', 'indices': [120, 125]}]
[{'text': 'VoteRedStraightTicket2018', 'indices': [52, 78]}]
[{'text': 'AkElect', 'indices': [65, 73]}]
[{'text': 'AZ07', 'indices': [51, 56]}]
[{'text': 'AsianAmerican', 'indices': [42, 56]}, {'text': 'MidtermElection', 'indices': [109, 125]}]
[{'text': 'potus45', 'indices': [77, 85]}, {'text': 'Kavanaugh', 'indices': [90, 100]}, {'text': 'Republic', 'indices': [118, 127]}, {'text': '1A', 'indices': [128, 131]}, {'text': '2

[{'text': 'CA04', 'indices': [102, 107]}]
[{'text': 'stephencolbert', 'indices': [41, 56]}]
[{'text': 'FARA', 'indices': [66, 71]}]
[{'text': 'Insurance', 'indices': [39, 49]}, {'text': 'PreExistingConditions', 'indices': [70, 92]}]
[{'text': 'MidtermsMatter', 'indices': [20, 35]}, {'text': 'KevinMcCarthy', 'indices': [50, 64]}]
[{'text': 'youareadumbass', 'indices': [12, 27]}, {'text': 'worthless', 'indices': [28, 38]}, {'text': 'loser', 'indices': [39, 45]}]
[{'text': 'loser', 'indices': [59, 65]}]
[{'text': 'CA04', 'indices': [118, 123]}]
[{'text': 'JimCosta', 'indices': [102, 111]}]
[{'text': 'Press', 'indices': [40, 46]}]
[{'text': 'CA04', 'indices': [84, 89]}]
[{'text': 'MilitaryTribunals', 'indices': [67, 85]}, {'text': 'OrangeSchiffyCoveralls', 'indices': [87, 110]}]
[{'text': 'CA04', 'indices': [118, 123]}]
[{'text': 'CA04', 'indices': [47, 52]}]
[{'text': 'JustKeepSwimming', 'indices': [114, 131]}]
[{'text': 'Congress', 'indices': [0, 9]}, {'text': 'Senate', 'indices': [10, 1

[{'text': 'TrulyTouchingAndUnbelievableHowTheWorldStandsByAndWatchTyrantBiyaSlaughterAmbazonians', 'indices': [33, 119]}]
[{'text': 'GOPDelete8', 'indices': [27, 38]}]
[{'text': 'CA04', 'indices': [48, 53]}, {'text': 'CA04', 'indices': [63, 68]}]
[{'text': 'JollyPOTUS2020', 'indices': [107, 122]}]
[{'text': 'CA01', 'indices': [88, 93]}]
[{'text': 'Democrats', 'indices': [70, 80]}]
[{'text': 'Honduras', 'indices': [16, 25]}, {'text': 'eeuu', 'indices': [131, 136]}]
[{'text': 'CA04', 'indices': [34, 39]}, {'text': 'TeamMorse', 'indices': [40, 50]}, {'text': 'Moms4Morse', 'indices': [51, 62]}]
[{'text': '5Days', 'indices': [86, 92]}, {'text': 'AL04', 'indices': [128, 133]}]
[{'text': 'OKeefed', 'indices': [97, 105]}]
[{'text': 'CloserNation', 'indices': [15, 28]}]
[{'text': 'magaterrostisfortrump', 'indices': [20, 42]}]
[{'text': 'CA01', 'indices': [90, 95]}, {'text': 'CA04', 'indices': [118, 123]}]
[{'text': 'CA04', 'indices': [65, 70]}, {'text': 'CA04', 'indices': [80, 85]}]
[{'text': '

[{'text': 'BringItHome', 'indices': [33, 45]}, {'text': 'FlipHouseBlue', 'indices': [84, 98]}]
[{'text': 'Trump', 'indices': [51, 57]}, {'text': 'LyingTed', 'indices': [70, 79]}]
[{'text': 'CA49', 'indices': [106, 111]}]
[{'text': 'HConRes138', 'indices': [33, 44]}]
[{'text': 'MarkLevin', 'indices': [41, 51]}]
[{'text': 'dg4az', 'indices': [23, 29]}, {'text': 'RedforEd', 'indices': [30, 39]}]
[{'text': 'CA49', 'indices': [94, 99]}]
[{'text': 'LauraSilsby', 'indices': [104, 116]}]
[{'text': 'VoteAntonio2018', 'indices': [44, 60]}, {'text': 'VenturaCounty', 'indices': [61, 75]}, {'text': 'MakeCaliforniaGreatAgain', 'indices': [76, 101]}]
[{'text': 'NoBrainer', 'indices': [40, 50]}]
[{'text': 'KennedyLive', 'indices': [22, 34]}]
[{'text': 'CA04', 'indices': [84, 89]}]
[{'text': 'FridayFeeling', 'indices': [62, 76]}, {'text': 'CaravanaMigrantes', 'indices': [77, 95]}, {'text': 'HondurasCaravan', 'indices': [96, 112]}]
[{'text': 'copolitics', 'indices': [105, 116]}, {'text': 'coleg', 'indic

[{'text': 'CA22', 'indices': [24, 29]}, {'text': 'RemoveNunes', 'indices': [49, 61]}, {'text': 'tRumpStooge', 'indices': [64, 76]}]
[{'text': 'Wisconsin', 'indices': [104, 114]}]
[{'text': 'Chicago', 'indices': [36, 44]}]
[{'text': 'VOTE', 'indices': [120, 125]}]
[{'text': 'MaxineWaters', 'indices': [45, 58]}]
[{'text': 'ReplaceNunes', 'indices': [6, 19]}, {'text': 'tRumpStooge', 'indices': [22, 34]}, {'text': 'JanzTeam', 'indices': [46, 55]}]
[{'text': 'FridayFollow', 'indices': [42, 55]}]
[{'text': 'CA49', 'indices': [59, 64]}, {'text': 'CA48', 'indices': [65, 70]}]
[{'text': 'Beto', 'indices': [46, 51]}, {'text': 'StaceyAbramsForGA', 'indices': [87, 105]}, {'text': 'AndrewGillum', 'indices': [110, 123]}]
[{'text': 'ReplaceNunes', 'indices': [25, 38]}, {'text': 'tRumpStooge', 'indices': [41, 53]}, {'text': 'JanzTeam', 'indices': [65, 74]}]
[{'text': 'VOTE', 'indices': [120, 125]}]
[{'text': 'Beto', 'indices': [46, 51]}, {'text': 'StaceyAbramsForGA', 'indices': [87, 105]}, {'text': 'A

...

[{'text': 'Trump', 'indices': [60, 66]}, {'text': 'nbcnews', 'indices': [73, 81]}, {'text': 'CBS', 'indices': [83, 87]}, {'text': 'CBSnews', 'indices': [88, 96]}, {'text': 'CNN', 'indices': [97, 101]}, {'text': 'NBCNEWS', 'indices': [102, 110]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'MaxineWaters', 'indices': [79, 92]}, {'text': 'revenge', 'indices': [124, 132]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'FLgov', 'indices': [100, 106]}, {'text': 'FLsen', 'indices': [107, 113]}]
[{'text': 'vote', 'indices': [0, 5]}]
[{'text': 'Democratic', 'indices': [34, 45]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'MSM', 'indices': [38, 42]}, {'text': 'Elections2018', 'indices': [107, 121]}]
[{'text': 'MaxineWaters', 'indices': [79, 92]}, {'text': 'revenge', 'indices': [124, 132]}]
[{'text': 'TedLieu', 'indices': [65, 73]}]
[{'text': 'Kha

[{'text': 'FridayFeeling', 'indices': [31, 45]}]
[{'text': 'FARA', 'indices': [66, 71]}]
[{'text': 'CaliforniaPatriots', 'indices': [17, 36]}]
[{'text': 'California', 'indices': [37, 48]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'CaliforniaPatriots', 'indices': [17, 36]}]
[{'text': 'GOPDelete8', 'indices': [27, 38]}]
[{'text': 'IranTerror', 'indices': [119, 130]}]
[{'text': 'stopthehate', 'indices': [125, 137]}]
[{'text': 'florida', 'indices': [31, 39]}, {'text': 'America', 'indices': [40, 48]}, {'text': 'republican', 'indices': [49, 60]}]
[{'text': 'IranTerror', 'indices': [119, 130]}]
[{'text': 'MSM', 'indices': [38, 42]}, {'text': 'Elections2018', 'indices': [107, 121]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'Elvis', 'indices': [49, 55]}, {'text': 'ElvisPresley', 'indices': [56, 69]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'t

[{'text': 'CA48', 'indices': [127, 132]}, {'text': 'GOTV', 'indices': [133, 138]}]
[{'text': 'Cameroon', 'indices': [19, 28]}]
[{'text': 'CA04', 'indices': [118, 123]}]
[{'text': 'SocialistHack', 'indices': [53, 67]}]
[{'text': 'DemsSoDiverse', 'indices': [17, 31]}]
[{'text': 'VoteWisely', 'indices': [22, 33]}, {'text': 'vote', 'indices': [56, 61]}]
[{'text': 'ResistandWin', 'indices': [12, 25]}]
[{'text': 'AZ01', 'indices': [36, 41]}]
[{'text': 'CA43', 'indices': [19, 24]}, {'text': 'CA43', 'indices': [55, 60]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'LastWord', 'indices': [54, 63]}]
[{'text': 'Beto', 'indices': [46, 51]}, {'text': 'StaceyAbramsForGA', 'indices': [87, 105]}, {'text': 'AndrewGillum', 'indices': [110, 123]}]
[{'text': 'Wisconsin', 'indices': [104, 114]}]
[{'text': 'CA50', 'indices': [92, 97]}, {'text': 'flipfive', 'indices': [115, 124]}]
[{'text': 'PreExistingConditions', 'indices': [67, 89]}]
[{'text': 'Republican

[{'text': 'CALIFORNIA', 'indices': [16, 27]}, {'text': 'CONSERVATIVES', 'indices': [28, 42]}]
[{'text': 'CALIFORNIA', 'indices': [16, 27]}, {'text': 'CONSERVATIVES', 'indices': [28, 42]}]
[{'text': 'Sisolak', 'indices': [51, 59]}, {'text': 'Laxalt', 'indices': [68, 75]}, {'text': 'RyanBundy4Governor', 'indices': [107, 126]}, {'text': 'Nevada', 'indices': [130, 137]}]
[{'text': 'PayAttention', 'indices': [19, 32]}, {'text': 'California', 'indices': [33, 44]}, {'text': 'Californians', 'indices': [45, 58]}, {'text': 'CA22', 'indices': [59, 64]}, {'text': 'Vote', 'indices': [72, 77]}, {'text': 'AndrewJanz', 'indices': [82, 93]}]
[{'text': 'AZ08', 'indices': [104, 109]}]
[{'text': 'MSM', 'indices': [38, 42]}, {'text': 'Elections2018', 'indices': [107, 121]}]
[{'text': 'Eritrea', 'indices': [20, 28]}]
[{'text': 'MSM', 'indices': [38, 42]}, {'text': 'Elections2018', 'indices': [107, 121]}]
[{'text': 'CALIFORNIA', 'indices': [16, 27]}, {'text': 'CONSERVATIVES', 'indices': [28, 42]}]
[{'text': 

[{'text': 'MaxineWaters', 'indices': [79, 92]}, {'text': 'revenge', 'indices': [124, 132]}]
[{'text': 'Amendment4', 'indices': [125, 136]}]
[{'text': 'boycottTheView', 'indices': [117, 132]}]
[{'text': 'CA22', 'indices': [47, 52]}, {'text': 'replaceNunes', 'indices': [53, 66]}]
[{'text': 'CA22', 'indices': [92, 97]}, {'text': 'ProudBoys', 'indices': [126, 136]}]
[{'text': 'CA48', 'indices': [39, 44]}]
[{'text': 'CaliforniaPatriots', 'indices': [17, 36]}]
[{'text': 'CALIFORNIA', 'indices': [16, 27]}, {'text': 'CONSERVATIVES', 'indices': [28, 42]}]
[{'text': 'CALIFORNIA', 'indices': [16, 27]}, {'text': 'CONSERVATIVES', 'indices': [28, 42]}]
[{'text': 'CA48', 'indices': [39, 44]}]
[{'text': 'PreExistingConditions', 'indices': [14, 36]}]
[{'text': 'VoteShariceDavidsToSaveHealthCare', 'indices': [76, 110]}]
[{'text': 'MSM', 'indices': [38, 42]}, {'text': 'Elections2018', 'indices': [107, 121]}]
[{'text': 'GOTV', 'indices': [48, 53]}, {'text': 'CA22', 'indices': [54, 59]}]
[{'text': 'CA', 'i

In [64]:
partisan_tweets_per_candidate

{'@kennedy4alabama': {'conservative': 0, 'liberal': 25, 'neutral': 23},
 '@repbyrne': {'conservative': 16, 'liberal': 0, 'neutral': 222},
 '@tabithak': {'conservative': 0, 'liberal': 81, 'neutral': 947},
 '@repmartharoby': {'conservative': 0, 'liberal': 0, 'neutral': 974},
 '@haganforhouse': {'conservative': 0, 'liberal': 150, 'neutral': 1536},
 '@repmikerogersal': {'conservative': 0, 'liberal': 0, 'neutral': 126},
 '@leeauman2018': {'conservative': 0, 'liberal': 0, 'neutral': 766},
 '@robert_aderholt': {'conservative': 0, 'liberal': 0, 'neutral': 1008},
 '@peter_joffrion': {'conservative': 0, 'liberal': 19, 'neutral': 3210},
 '@repmobrooks': {'conservative': 0, 'liberal': 0, 'neutral': 1808},
 '@votedannerkline': {'conservative': 0, 'liberal': 8, 'neutral': 418},
 '@usrepgarypalmer': {'conservative': 16, 'liberal': 0, 'neutral': 526},
 '@repterrisewell': {'conservative': 0, 'liberal': 18, 'neutral': 172},
 '@repdonyoung': {'conservative': 32, 'liberal': 0, 'neutral': 1382},
 '@alysega